# Cleaning
- renommer des colonnes (df.rename(columns=....))
- trouver/supprimer les données dupliquées (.duplicated / .drop_duplicates)
- trouver les NA (df.column.isna() / df.column.notna())
- remplacer les NA (.fillna())
- remplacer n'importe quelle valeur (.replace({OLD_VALUE: NEW_VALUE, …}))
- changer le type d'une série (aka cast) (.astype(type) / pd.to_numeric / pd.to_datetime)
- fallback les valeurs NA d'une colonne sur une autre colonne: combine_first

### .dt accessor (for date-type columns)
→ https://pandas.pydata.org/pandas-docs/stable/reference/series.html#datetime-properties

### .str accessor (for string-typed columns)

→ https://pandas.pydata.org/pandas-docs/stable/reference/series.html#string-handling

### Regexes:
- cheat sheet: https://www.debuggex.com/cheatsheet/regex/python
- talk sympa: https://www.youtube.com/watch?v=abrcJ9MpF60
- le module dédié "re" de python: https://docs.python.org/3/library/re.html
- site web pour tester des regexes: https://regex101.com

In [28]:
import pandas as pd
import json
import requests
import time
from requests import get

In [29]:
# if you don't want pandas.read_csv to mess with data types,
# you can force it to keep str values by specifying dtype=str.
people = pd.read_csv('people.csv')

In [30]:
people.shape

(209, 15)

In [31]:
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 15 columns):
id               209 non-null int64
first_name       207 non-null object
last_name        207 non-null object
email address    203 non-null object
gender           207 non-null object
age              207 non-null object
money            190 non-null object
lon              207 non-null float64
lat              207 non-null float64
phone            83 non-null object
registration     207 non-null object
inactive         207 non-null object
last_seen        190 non-null float64
address          207 non-null object
preference       207 non-null object
dtypes: float64(3), int64(1), object(11)
memory usage: 24.6+ KB


In [32]:
people.gender.unique()

array(['Female', 'Male', 'F', 'M', nan], dtype=object)

In [33]:
def clean_people(df):
    # rename columns:
    df = df.rename(columns={'email address': 'email'})
    
    # remove rows which have an empty "first_name" (NA):
    #df = df[df.first_name.notna()] <- equivalent to next line:
    df = df.dropna(subset=['first_name'])
    
    # drop duplicates on ID column:
    df = df.drop_duplicates()
    
    # Normalize gender column:
    df['gender'] = df['gender'].replace({'Female': 'F', 'Male': 'M'})
    
    # Convert column "age" to number (coerce: put NaN for bad values):
    df['age'] = pd.to_numeric(df.age, errors='coerce')
    
    # Convert columns to date type:
    df['registration'] = pd.to_datetime(df.registration)
    df['last_seen'] = pd.to_datetime(df.last_seen, unit='s')
    # When missing, last seen should fallback to the registration date:
    df['last_seen'] = df.last_seen.combine_first(df.registration)
    
    # Add a "full_name" column by concatenating two other ones:
    df['full_name'] = df.first_name + " " + df.last_name
    
    # Add a "country" column by extracting it from the address, with a split:
    df['country'] = df.address.str.split(', ').str[1]
    
    # Column "money" contains values like "$50.23" or "€23,09".
    # We want to make it uniform (only dollar currency) and as number, not str.
    df['currency'] = df.money.str[0]  # extract first char ($/€) to a new "currency" column
    df['money'] = df.money.str[1:].str.replace(',', '.')  # extract remaining chars and replace , by .
    df['money'] = pd.to_numeric(df.money)  # convert to number
    # convert euros cells to dollar:
    df.loc[df.currency == '€', 'money'] = df[df.currency == '€'].money * 1.10
    del df['currency']  # remove "currency" column which is now useless
    
    # Keep only rows where email is not NA:
    df = df.dropna(subset=['email'])
    # Keep only rows where email is a good email:
    # CAUTION: in the real world you should not use dummy regexes like this to validate email addresses,
    # but instead use a dedicated tool like https://github.com/syrusakbary/validate_email.
    df = df[df.email.str.contains('.+@[0-9a-zA-Z\.\-_]+\.\w{2,}')]
    # Some users may use email alias (example: john.smith+truc@gmail.com is an alias for john.smith@gmail.com).
    # We want to drop these duplicates. To do that, we extract the 'alias' part with a regex:
    groups = df.email.str.extract('([0-9a-zA-Z\.\-_]+)(\+[0-9a-zA-Z\.\-_]+)?(@[0-9a-zA-Z\.\-_]+\.\w{2,})')
    df['email'] = groups[0] + groups[2]  # we override the email with the email without the alias part
    # Then, just use drop_duplicates, which will keep the first line by default:
    df = df.drop_duplicates(subset=['email'])
    
    return df

df_clean = clean_people(people)

In [34]:
df_clean

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic
1,27570,Ruthi,Ross,rross3w@sohu.com,F,57.0,20.370,19.230220,50.466575,NaN,2018-10-23,False,2019-08-30 11:41:04,"Siewierz, Poland",entrée/plat/dessert,Ruthi Ross,Poland
2,27572,Silas,Stourton,silas.stourton3y@answers.com,M,22.0,36.289,118.831081,24.984813,NaN,2018-12-30,False,2019-03-27 13:10:32,"Longbo, China",entrée/plat/dessert,Silas Stourton,China
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64.0,108.823,121.648987,-8.844744,NaN,2019-03-12,False,2019-03-12 00:00:00,"Potulando, Indonesia",entrée/plat,Roxine Pettecrew,Indonesia
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,27465,Adelind,Christin,achristinz@blogs.com,F,30.0,90.024,41.427853,52.415968,NaN,2019-06-04,False,2019-06-04 00:00:00,"Znamenka, Russia",entrée/plat/dessert,Adelind Christin,Russia
205,27476,Petronella,Pickance,ppickance1a@uiuc.edu,F,28.0,40.865,132.702111,33.762296,NaN,2017-11-22,False,2018-07-29 13:18:29,"Iyo, Japan",entrée/plat/dessert,Petronella Pickance,Japan
206,27521,Moritz,Issacof,missacof2j@wired.com,M,21.0,91.970,72.311099,31.263396,0298949280,2018-08-20,False,2019-02-10 16:39:03,"Jhang Sadr, Pakistan",entrée/plat/dessert,Moritz Issacof,Pakistan
207,27441,Quintana,Foulstone,qfoulstoneb@newsvine.com,F,32.0,61.204,111.138442,-6.783653,0734595126,2018-10-25,False,2018-10-25 00:00:00,"Mantingantengah, Indonesia",entrée/plat/dessert,Quintana Foulstone,Indonesia


In [35]:
df_clean['last_seen']=pd.to_datetime(df_clean['last_seen'])

In [36]:
df_clean

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic
1,27570,Ruthi,Ross,rross3w@sohu.com,F,57.0,20.370,19.230220,50.466575,NaN,2018-10-23,False,2019-08-30 11:41:04,"Siewierz, Poland",entrée/plat/dessert,Ruthi Ross,Poland
2,27572,Silas,Stourton,silas.stourton3y@answers.com,M,22.0,36.289,118.831081,24.984813,NaN,2018-12-30,False,2019-03-27 13:10:32,"Longbo, China",entrée/plat/dessert,Silas Stourton,China
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64.0,108.823,121.648987,-8.844744,NaN,2019-03-12,False,2019-03-12 00:00:00,"Potulando, Indonesia",entrée/plat,Roxine Pettecrew,Indonesia
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,27465,Adelind,Christin,achristinz@blogs.com,F,30.0,90.024,41.427853,52.415968,NaN,2019-06-04,False,2019-06-04 00:00:00,"Znamenka, Russia",entrée/plat/dessert,Adelind Christin,Russia
205,27476,Petronella,Pickance,ppickance1a@uiuc.edu,F,28.0,40.865,132.702111,33.762296,NaN,2017-11-22,False,2018-07-29 13:18:29,"Iyo, Japan",entrée/plat/dessert,Petronella Pickance,Japan
206,27521,Moritz,Issacof,missacof2j@wired.com,M,21.0,91.970,72.311099,31.263396,0298949280,2018-08-20,False,2019-02-10 16:39:03,"Jhang Sadr, Pakistan",entrée/plat/dessert,Moritz Issacof,Pakistan
207,27441,Quintana,Foulstone,qfoulstoneb@newsvine.com,F,32.0,61.204,111.138442,-6.783653,0734595126,2018-10-25,False,2018-10-25 00:00:00,"Mantingantengah, Indonesia",entrée/plat/dessert,Quintana Foulstone,Indonesia


In [37]:
 from datetime import datetime, timedelta

In [38]:
present = datetime.now()

In [39]:
past = datetime.now() - timedelta(days=365)

In [40]:
def statuts(lastdate):
    
    if lastdate<past:
        return False
    else:
        return True

  
    

In [41]:
df_clean.inactive = df_clean.last_seen.apply(statuts)

In [42]:
df_clean

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,True,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic
1,27570,Ruthi,Ross,rross3w@sohu.com,F,57.0,20.370,19.230220,50.466575,NaN,2018-10-23,True,2019-08-30 11:41:04,"Siewierz, Poland",entrée/plat/dessert,Ruthi Ross,Poland
2,27572,Silas,Stourton,silas.stourton3y@answers.com,M,22.0,36.289,118.831081,24.984813,NaN,2018-12-30,True,2019-03-27 13:10:32,"Longbo, China",entrée/plat/dessert,Silas Stourton,China
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64.0,108.823,121.648987,-8.844744,NaN,2019-03-12,True,2019-03-12 00:00:00,"Potulando, Indonesia",entrée/plat,Roxine Pettecrew,Indonesia
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,True,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,27465,Adelind,Christin,achristinz@blogs.com,F,30.0,90.024,41.427853,52.415968,NaN,2019-06-04,True,2019-06-04 00:00:00,"Znamenka, Russia",entrée/plat/dessert,Adelind Christin,Russia
205,27476,Petronella,Pickance,ppickance1a@uiuc.edu,F,28.0,40.865,132.702111,33.762296,NaN,2017-11-22,False,2018-07-29 13:18:29,"Iyo, Japan",entrée/plat/dessert,Petronella Pickance,Japan
206,27521,Moritz,Issacof,missacof2j@wired.com,M,21.0,91.970,72.311099,31.263396,0298949280,2018-08-20,True,2019-02-10 16:39:03,"Jhang Sadr, Pakistan",entrée/plat/dessert,Moritz Issacof,Pakistan
207,27441,Quintana,Foulstone,qfoulstoneb@newsvine.com,F,32.0,61.204,111.138442,-6.783653,0734595126,2018-10-25,True,2018-10-25 00:00:00,"Mantingantengah, Indonesia",entrée/plat/dessert,Quintana Foulstone,Indonesia


In [43]:
df_clean=df_clean.dropna(axis=0,subset=['phone'])

In [44]:
df_clean

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,True,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,True,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic
12,27548,Stanley,McCullen,smccullen3a@timesonline.co.uk,M,60.0,NaN,119.141473,28.074649,0875503094,2017-06-05,False,2017-06-05 00:00:00,"Longquan, China",entrée/plat/dessert,Stanley McCullen,China
15,27619,Shelley,Skeemor,sskeemor59@princeton.edu,F,44.0,41.500,111.878471,-7.584917,0547556034,2019-01-24,True,2019-05-04 13:18:24,"Guyangan, Indonesia",entrée/plat/dessert,Shelley Skeemor,Indonesia
16,27442,Mort,Midford,mmidfordc@yale.edu,M,59.0,NaN,6.848501,5.959253,0811321968,2019-06-25,True,2019-06-25 00:00:00,"Ozubulu, Nigeria",entrée/plat,Mort Midford,Nigeria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,27624,Dur,Hallaways,dhallaways5e@delicious.com,M,25.0,76.076,106.720976,28.739865,0317009357,2019-07-27,True,2019-09-23 05:01:36,"Ganshui, China",entrée/plat/dessert,Dur Hallaways,China
199,27604,Michelina,Lowcock,mlowcock4u@nba.com,F,42.0,NaN,12.325710,58.335399,0276520906,2017-07-12,False,2017-07-12 00:00:00,"Vänersborg, Sweden",entrée/plat/dessert,Michelina Lowcock,Sweden
206,27521,Moritz,Issacof,missacof2j@wired.com,M,21.0,91.970,72.311099,31.263396,0298949280,2018-08-20,True,2019-02-10 16:39:03,"Jhang Sadr, Pakistan",entrée/plat/dessert,Moritz Issacof,Pakistan
207,27441,Quintana,Foulstone,qfoulstoneb@newsvine.com,F,32.0,61.204,111.138442,-6.783653,0734595126,2018-10-25,True,2018-10-25 00:00:00,"Mantingantengah, Indonesia",entrée/plat/dessert,Quintana Foulstone,Indonesia


In [45]:
df_clean = df_clean[df_clean.phone.str.contains('0[6|7]{1}[0-9]{8}')]

In [46]:
df_clean

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country
27,27551,Brigg,Bryett,bbryett3d@prlog.org,M,20.0,28.050,102.638296,24.301245,0746863267,2017-10-25,True,2019-04-10 01:46:26,"Dayingjie, China",entrée/plat/dessert,Brigg Bryett,China
45,27432,Gardener,Kempson,gkempson2@furl.net,M,33.0,41.300,25.911060,49.103782,0774378059,2017-09-08,True,2019-03-18 18:58:59,"Kopychyntsi, Ukraine",entrée/plat/dessert,Gardener Kempson,Ukraine
49,27458,Osborne,Donn,odonns@webeden.co.uk,M,33.0,37.609,117.430061,23.701262,0721208335,2018-09-01,True,2019-03-07 22:09:24,"Dongshe, China",plat/dessert,Osborne Donn,China
68,27541,Clio,Anning,canning33@globo.com,F,37.0,95.249,29.291635,-26.215504,0744215909,2018-01-14,True,2019-01-28 08:34:53,"Kriel, South Africa",entrée/plat/dessert,Clio Anning,South Africa
70,27499,Bret,Bradnock,bbradnock1x@reference.com,M,54.0,28.721,39.531427,44.418695,0644377486,2018-04-27,False,2018-06-27 07:42:59,"Khadyzhensk, Russia",entrée/plat/dessert,Bret Bradnock,Russia
81,27628,Arliene,Sedgeman,asedgeman5i@cmu.edu,F,33.0,67.639,14.521628,59.321605,0795796664,2017-05-10,False,2018-07-09 01:45:46,"Karlskoga, Sweden",entrée/plat/dessert,Arliene Sedgeman,Sweden
92,27438,Anson,Churchward,achurchward8@mit.edu,M,45.0,81.450,-9.485174,52.446080,0665664983,2019-07-17,True,2019-07-17 00:00:00,"Listowel, Ireland",entrée/plat/dessert,Anson Churchward,Ireland
154,27594,Ada,McAvaddy,amcavaddy4k@storify.com,F,54.0,48.120,112.528308,32.990664,0771644155,2019-04-24,True,2019-06-06 07:42:21,"Nanyang, China",entrée/plat/dessert,Ada McAvaddy,China
171,27629,Lannie,Bimson,lbimson5j@ifeng.com,M,54.0,69.550,110.325536,-7.874818,0735950711,2017-07-10,True,2019-08-02 17:34:45,"Bantul, Indonesia",entrée/plat/dessert,Lannie Bimson,Indonesia
172,27603,Adara,Farragher,afarragher4t@foxnews.com,F,61.0,NaN,29.741025,46.508480,0606471647,2018-03-12,False,2018-03-12 00:00:00,"Slobozia, Moldova",entrée/plat/dessert,Adara Farragher,Moldova


In [54]:
#Apres utiliser locationIQ API
